# Canvas Backup

In [ ]:
import http.server
import aiohttp
import ipywidgets as widgets
import asyncio
import pathlib
import canvas
from urllib.parse import urljoin

In [ ]:
token = input("Enter token")
headers = {"Authorization": f"Bearer {token}"}

## Create authorized `ClientSession`

In [ ]:
session = aiohttp.ClientSession(headers=headers, timeout=aiohttp.ClientTimeout(None))

In [ ]:
class RateLimitProxy:
    def __init__(self, session, n_requests=10):
        self._session = session
        self._semaphore = asyncio.BoundedSemaphore(n_requests)
        
    def __getattr__(self, name):
        return getattr(self._session, name)
    
    async def get(self, *args, **kwargs):
        async with self._semaphore:
            return await self._session.get(*args, **kwargs)
        
session = RateLimitProxy(session, 40)

## Get a session URL for selenium

In [ ]:
response = await session.get(urljoin(canvas.DOMAIN_URL, "login/session_token"))
assert response.status == 200
session_url = (await response.json())['session_url']
session_url

In [ ]:
response = await session.get(urljoin(canvas.API_URL, "courses"), data={'per_page': 200})
courses = await response.json()
print(f"Found {len(courses)} courses.")

## Support pagination

In [ ]:
courses = await canvas.get_courses(session)
id_to_course = {c['id']: c for c in courses}

In [ ]:
course_select = widgets.SelectMultiple(
    options=[c['name'] for c in courses],
    description='Courses',
    disabled=False,
    layout=widgets.Layout(width='50%', height='20em')
)
course_select

In [ ]:
selected_courses = [courses[i] for i in course_select.index]

In [ ]:
downloads_dir = pathlib.Path.cwd() / "canvas"
downloads_dir.mkdir(exist_ok=True)

In [ ]:
await canvas.download_course(session, downloads_dir, selected_courses[0])

In [ ]:
tasks = [asyncio.create_task(canvas.download_course(session, downloads_dir, c)) for c in selected_courses]    
tasks

In [ ]:
asyncio.all_tasks()

Now launch http.server with appropriate directory